In [1]:

import numpy as np
import h5py
import scipy.io
np.random.seed(1337) # for reproducibility
import matplotlib.pyplot as plt

from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D,Convolution2D, MaxPooling1D
from keras.regularizers import l2, activity_l1
from keras.constraints import maxnorm
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras.optimizers import Adam, SGD



/Users/jx/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using Theano backend.


In [2]:
import numpy as np
import h5py
import scipy.io
validmat = scipy.io.loadmat('valid3D.mat')
testmat = scipy.io.loadmat('test3D.mat')
trainmat = scipy.io.loadmat('train3D.mat')


X_train = np.transpose(np.array(trainmat['tr'][0][0][0]),axes=(0,2,1))
# X_train = np.expand_dims(X_train,axis=(2))
y_train = np.array(trainmat['tr'][0][0][1]).squeeze()
# y_train = y_train.reshape((-1, 1))
# y_train.squeeze()
print X_train.shape
print y_train.shape



(22059, 8, 60)
(22059,)


In [8]:


lr = 1e-6#learning rate
reg = 1e-3
print 'building model'
nb_filters=32
model = Sequential()
model.add(LSTM(4,  W_regularizer=l2(reg),return_sequences=True, input_shape=(8, 60)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
# model.add(LSTM(64,  W_regularizer=l2(reg),return_sequences=True)) # return sequences is needed for stacking
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(LSTM(128,  W_regularizer=l2(reg)))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(Dense(30))
# model.add(Flatten( input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])))

# model.add(Dense(10))
# model.add(Dense(1))

# model.add(LSTM(32,24, W_regularizer=l2(reg),input_shape=(X_train.shape[0], X_train.shape[1], X_train.shape[2]))) # adding conv layer collapses output
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
model.add(Convolution1D(4,12, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
model.add(Activation('tanh'))
model.add(Dropout(0.2))
model.add(Convolution1D(8,12, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
model.add(Activation('tanh'))
model.add(Dropout(0.2))





model.add(Flatten())


# model.add(Dense(30))
model.add(Dense(1))
adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)


print 'compiling model'
model.compile(loss='mse', optimizer='adam', metrics=["mse"])



checkpointer = ModelCheckpoint(filepath="bestmodel3D.hdf5", verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

X_valid=np.transpose(validmat['v'][0][0][0],axes=(0,2,1))
# X_valid=np.expand_dims(X_valid)
y_valid=np.array(validmat['v'][0][0][1]).squeeze()


model.fit(X_train, y_train, batch_size=256, nb_epoch=500, shuffle=True, validation_data=(X_valid, y_valid),callbacks=[checkpointer,earlystopper])



building model
compiling model
Train on 22059 samples, validate on 2758 samples
Epoch 1/500
22059/22059 [==============================] - 0s - loss: 0.0552 - mean_squared_error: 0.0296 - val_loss: 0.0160 - val_mean_squared_error: 0.0160
Epoch 2/500
22059/22059 [==============================] - 0s - loss: 0.0308 - mean_squared_error: 0.0178 - val_loss: 0.0157 - val_mean_squared_error: 0.0157
Epoch 3/500
22059/22059 [==============================] - 0s - loss: 0.0237 - mean_squared_error: 0.0172 - val_loss: 0.0157 - val_mean_squared_error: 0.0157
Epoch 4/500
22059/22059 [==============================] - 0s - loss: 0.0203 - mean_squared_error: 0.0168 - val_loss: 0.0158 - val_mean_squared_error: 0.0158
Epoch 5/500
22059/22059 [==============================] - 0s - loss: 0.0187 - mean_squared_error: 0.0167 - val_loss: 0.0158 - val_mean_squared_error: 0.0158
Epoch 6/500
22059/22059 [==============================] - 0s - loss: 0.0180 - mean_squared_error: 0.0167 - val_loss: 0.0158 - val

In [4]:
%matplotlib

out=model.predict(X_train, batch_size=512,verbose=1)
plt.plot(y_train,out,'ro')

model.summary()


Using matplotlib backend: MacOSX
22016/22059 [============================>.] - ETA: 0s____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 8, 64)         32000       lstm_input_1[0][0]               
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 8, 64)         0           lstm_1[0][0]                     
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 8, 64)         0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)  (None, 8, 16)         16400       dropout_1[0][0]                  
____

In [7]:
%matplotlib
X_test=np.transpose(testmat['tt'][0][0][0],axes=(0,2,1))
y_test=np.array(testmat['tt'][0][0][1]).squeeze()

out=model.predict(X_test, batch_size=512,verbose=1)
plt.plot(y_test,out,'ro')

model.summary()

Using matplotlib backend: MacOSX
2757/2757 [==============================] - 0s     
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_2 (LSTM)                    (None, 8, 64)         32000       lstm_input_2[0][0]               
____________________________________________________________________________________________________
activation_4 (Activation)        (None, 8, 64)         0           lstm_2[0][0]                     
____________________________________________________________________________________________________
dropout_4 (Dropout)              (None, 8, 64)         0           activation_4[0][0]               
____________________________________________________________________________________________________
convolution1d_3 (Convolution1D)  (None, 8, 16)         16400       dropout_4[0][0]                  
_____